Check the status of tube lines

http://docs.python-requests.org/en/master/

http://docs.python-requests.org/en/master/user/quickstart/#make-a-request Tutorial

https://github.com/timcnicholls/transport_api_demo/blob/master/harwell_wantage_bus.py

23-6 Removed Goodness attribute as just duplicates state

In [1]:
import requests
import json
import pprint
import unittest
import sys

In [2]:
def print_json(json_data):
    pprint.PrettyPrinter().pprint(json_data)

In [3]:
lines = ['bakerloo',
 'central',
 'circle',
 'district',
 'hammersmith-city',
 'jubilee',
 'metropolitan',
 'northern',
 'piccadilly',
 'victoria',
 'waterloo-city']

# Basic function of API

In [4]:
lin_num = 0 
print(lines[lin_num])

url = 'https://api.tfl.gov.uk/line/{}/status'.format(lines[lin_num])
response = requests.get(url)  # get the response object

bakerloo_data = response.json()[0]['lineStatuses']       # data is a list of statuses on the line
print_json(bakerloo_data)

bakerloo
[{'$type': 'Tfl.Api.Presentation.Entities.LineStatus, '
           'Tfl.Api.Presentation.Entities',
  'created': '0001-01-01T00:00:00',
  'id': 0,
  'statusSeverity': 10,
  'statusSeverityDescription': 'Good Service',
  'validityPeriods': []}]


In [5]:
statuses = [status['statusSeverityDescription'] for status in bakerloo_data]
statuses

['Good Service']

## Get list of statuses
Statuses is a list of statuses on a line

In [6]:
def get_statuses(data):
    """Get a list of all statuses on a line."""
    return [status['statusSeverityDescription'] for status in data]

get_statuses(bakerloo_data)

['Good Service']

# Get state
State is a string capturing the overall state of a line. Return 'Good status' if good, else a string of individual states seperated with a +

In [7]:
def get_state(statuses):
    """Return the overall state of a line."""
    if 'Good Service' in statuses:   # if good status, this is the only status returned
        return 'Good Service'
    else:
        return ' + '.join(set(statuses)) 

get_state(['Part Closure', 'Minor Delays'])

'Part Closure + Minor Delays'

# Get description
Return the list of reasons for the not good state

In [8]:
def get_description(state, data):
    """Return a description of the resons for the state of a line."""
    if state == 'Good service':   # if good status, this is the only status returned
        return ['Nothing to report']
    else:
        return [status['reason'] for status in data]

## Create unittests

In [19]:
## Mock data
mock_good_data = [{'statusSeverityDescription':'Good Service'}]

mock_bad_data = [{'statusSeverityDescription':'Part Closure', 
                                     'reason': 'investigations'},
                 {'statusSeverityDescription':'Minor Delays', 
                                     'reason': 'leaves'}]

mock_bad_statuses = ['Part Closure', 'Minor Delays']

class TestStatus(unittest.TestCase):
    def test_of_get_statuses(self):
        self.assertEqual(get_statuses(mock_good_data), ['Good Service'])
        self.assertEqual(get_statuses(mock_bad_data), mock_bad_statuses)
        
    def test_of_get_state(self):
        self.assertEqual(get_state(mock_bad_statuses), 'Part Closure + Minor Delays')
        
    def test_of_get_description(self):
        self.assertEqual(get_description('Good service', mock_good_data), ['Nothing to report'])
        self.assertEqual(get_description(mock_bad_statuses, mock_bad_data), ['investigations', 'leaves'])
        
unittest.main(argv=['ignored', '-v'], exit=False)

test_of_get_description (__main__.TestStatus) ... ok
test_of_get_state (__main__.TestStatus) ... ok
test_of_get_statuses (__main__.TestStatus) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


# Create tube class

Wrap in a class with a method for updating and another for returning line of interest. Follow https://github.com/timcnicholls/home-assistant/blob/transport-api/homeassistant/components/sensor/uk_transport.py

In [34]:
class LondonTubeSensor():    # Entity
    """
    Sensor that reads the status of a tube lines using the TFL API.
    """
    API_URL_BASE = "https://api.tfl.gov.uk/line/{}/status"
    ICON = 'mdi:subway'

    def __init__(self, name):
        """Initialize the sensor."""
        self._name = name             # the name of the line from the allowed list
        self._data = {}
        self._url = self.API_URL_BASE
        self._state = 'Updating'
        self._statuses = []
        self._description = ['Updating']

    @property
    def name(self):
        """Return the line name of the sensor."""
        return self._name

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON

    #@property
    #def unit_of_measurement(self):    # Do I need?
    #    """Return the unit this state is expressed in."""
    #    return ""
    
    @property
    def device_state_attributes(self):
        """Return other details about the sensor state."""
        attrs = {}  # {'attribution': 'Data provided by transportapi.com'}
        attrs['Goodness'] = self._goodness # if there is data, append
        attrs['Statuses'] = self._statuses # if there is data, append
        attrs['Description'] = self._description # if there is data, append
        return attrs

    def update(self):
        """Perform an API request and update the sensor."""
        
        response = requests.get(self._url.format(self._name.lower()))    # make a request
        print(response)
        
        if response.status_code != 200:
            #_LOGGER.warning("Invalid response from API")
            print("Invalid response from API")
            
        else:    
            self._data = response.json()[0]['lineStatuses']   # convert to json and get statuses list          
            self._statuses = self.get_statuses()
            self._state = self.get_state()
         #   self._description = self.get_description()

            
    def get_statuses(self):
        """Get a list of all statuses on a line."""
        return [status['statusSeverityDescription'] for status in self._data]
    
    def get_state(self):
        """Return the overall state of a line."""
        if 'Good Service' in self._statuses:   # if good status, this is the only status returned
            return 'Good Service'
        else:
            return ' + '.join(set(self._statuses)) 
        
    def get_description(self):
        """Return a description of the resons for the state of a line."""
        if self._state == 'Good service':   # if good status, this is the only status returned
            return ['Nothing to report']
        else:
            return [status['reason'] for status in self._data]

In [35]:
Bakerloo_sensor = LondonTubeSensor('Bakerloo')

In [38]:
#Bakerloo_sensor.update()
Bakerloo_sensor._data

[{'$type': 'Tfl.Api.Presentation.Entities.LineStatus, Tfl.Api.Presentation.Entities',
  'created': '0001-01-01T00:00:00',
  'id': 0,
  'statusSeverity': 10,
  'statusSeverityDescription': 'Good Service',
  'validityPeriods': []}]

# OLD

In [10]:
class LondonTubeSensorDEPRECATED():    # Entity
    """
    Sensor that reads the status of a tube lines using the TFL API.
    """

    API_URL_BASE = "https://api.tfl.gov.uk/line/{}/status"
    ICON = 'mdi:subway'

    def __init__(self, line):
        """Initialize the sensor."""
        self._data = {}
        self._url = self.API_URL_BASE
        self._line = line
        self._state = 'Updating'
        self._statuses = []
        self._goodness = False
        self._description = 'Updating'

    @property
    def name(self):
        """Return the line name of the sensor."""
        return self._line

    @property
    def state(self):
        """Return the state of the sensor."""
        return self._state

    @property
    def icon(self):
        """Icon to use in the frontend, if any."""
        return self.ICON

    @property
    def unit_of_measurement(self):    # Do I need?
        """Return the unit this state is expressed in."""
        return ""

    def update(self):
        """Perform an API request."""

        try:
            response = requests.get(self._url.format(self._line.lower()))
            response.raise_for_status()
            self._data = response.json()[0]['lineStatuses']   # convert to json and get statuses list
            self._statuses = [status['statusSeverityDescription'] for status in self._data]

            if 'Good Service' in self._statuses:   # if good status, this is the only status returned
                self._state = 'Good Service'
                self._goodness = True              # convenience attribute to detect good service
                self._description = 'Nothing to report'
            else:
                self._state = ' + '.join(set(self._statuses))   # get the unique statuses and join
                self._goodness = False
                self._description = [status['reason'] for status in self._data] # get the reasons

        except requests.RequestException as req_exc:
            print(
                'Invalid response from API: %s', req_exc
            )

    @property
    def device_state_attributes(self):
        """Return other details about the sensor state."""
        attrs = {}  # {'attribution': 'Data provided by transportapi.com'}
        attrs['Goodness'] = self._goodness # if there is data, append
        attrs['Statuses'] = self._statuses # if there is data, append
        attrs['Description'] = self._description # if there is data, append
        return attrs
